# TDA (Topological Data Analysis)


# Main Imports

In [1]:
import pymongo
import pandas as pd
#import Production_Nikkei.tools as tools

# Get S&P componants

Pour cet exemple de **TDA** nous allons crééer un cluster d'actions liées entre elles grace à l'analyse de leurs returns

Dans un premier temps, extrayons les composants du S&P

In [2]:
def list_index_constituant(path = "./sp_components.txt",from_base=1):
    fd = open(path, 'r')
    index = list(filter(lambda x : x,fd.read().split('\n')))
    fd.close()

    return index

spy_list = list_index_constituant()

In [ ]:
spy_list = sorted(spy_list)
s = spy_list[:10]
print(s)

Récuperrons les daily returns de ces tickers

In [4]:
res = pd.read_csv("./sp_value.csv",index_col=0)

In [ ]:
res

In [ ]:
res

In [7]:
sp_data = res[res.field == 'daily_return']

On fait pivoter le dataFrame afn d'avoir les tickers en header

In [8]:
sp_data = sp_data[["ticker", 'value']].pivot( columns='ticker', values = 'value')

In [ ]:
sp_data

In [10]:
targets = ['AAPL US Equity', 'MSFT US Equity', 'GOOG US Equity']

data_features = sp_data.copy()

In [11]:
data_features = data_features.dropna(axis=1)

# LOG RETURNS



# <span style="color:yellow; font-weight:bold">PCA + UMAP</span>

## <span style="color:orange; font-weight:bold">PCA</span>

### Pourquoi utiliser PCA ?
- Pour éliminer le bruit.
- Pour simplifier les données.
- Pour garder uniquement l’information avec le plus de variance.

### Que fait PCA(n_components=0.8) ?
- Il conserve les composantes principales qui expliquent 80% de la variance totale.
- Au lieu de 1000 dimensions (1 par jour), on se retrouve peut-être avec 10 ou 20 composantes principales, linéaires, non redondantes.

### Exemple
Imaginons que nous avionss 1000 log returns → PCA garde peut-être 15 directions principales :
- log_returns: (200 stocks x 1000 jours)
- reduced:     (200 stocks x ~15 composantes principales)

## <span style="color:orange; font-weight:bold">UMAP</span>

### Pourquoi UMAP après PCA ?
PCA est linéaire. Il ne détecte pas bien les formes non linéaires.

- UMAP capture la structure topologique : forme des nuages de points, distances locales et globales, etc.
- UMAP projette les données en 2D ou 3D, tout en respectant la forme intrinsèque de l’espace original.

### Que fait UMAP(n_components=2) ?
- Il projette les données PCA réduites dans un espace 2D.
- Chaque stock devient un point dans un plan, qui reflète sa similarité topologique avec les autres.

In [ ]:
from sklearn.decomposition import PCA
import umap
import pandas as pd


# Il conserve les composantes principales qui expliquent 80% de la variance totale.

# Au lieu de 1000 dimensions (1 par jour), on se retrouve peut-être avec 10 ou 20 composantes principales, linéaires, non redondantes.
pca = PCA(n_components=0.8, svd_solver='full', random_state=42)


reduced = pca.fit_transform(data_features.T)

# Avec UMAP on passe d'une 10aine de dimensions a juste 2 Dimensions
umap_model = umap.UMAP(n_components=2, random_state=42)
embedding = umap_model.fit_transform(reduced)

`embedding` contiennt le tableau qui contient les stocks transofrmés en points. Pour rappel, les dimensions ont été réduites à deux (X, Y) et chaque stock peut maintenant être assimilié à un point.

Sa proximité avec d'autres stocks peut traduire une relation interessante (corrlation implicite)

In [ ]:
data_features.T

Des formes et des **amas naturels** (clusters) apparaissent

A partir de ces coordonnées, on contruira des **clusters** avec `DBSCAN` , puis un **graphe topologique** avec `Mapper`

# Clusters with DBSCAN

In [ ]:
import kmapper as km
from sklearn.cluster import DBSCAN

# le mapper est l’objet principal qui va :
#   projeter, découper et organiser les données,
#   construire un graphe qui approxime la "forme" de la donnée.

mapper = km.KeplerMapper(verbose=1)

projected_data = embedding
graph = mapper.map(
    projected_data,
    data_features.T,
    cover=km.Cover(n_cubes=10, perc_overlap=0.5),
    clusterer=DBSCAN(metric='correlation', eps=0.3, min_samples=3)
)

la fonction `map` de notre **mapper** prends en arguments:
- `projected_data` qui sont les coordonnées en 2D de chaque ticker
- `log_returns.T` les returns avec les tickers en index
- `clusterer=DBSCAN`et `cover=km.Cover()` seront expliqués en bas

<span style="color:orange; font-weight:bold">Covering: </span>
L'argument `cover=km.Cover(n_cubes=10, perc_overlap=0.5)`divise l’espace 2D en grilles carrées (cubes) : ici 10x10

Chaque "cube" est une sous-région de l’espace projeté.

`n_cubes=10`: l'espace est divisé en 10x10 cubes
`perc_overlap=0.5` : les cubes se chevauchent à 50%. Cela permet d’avoir des clusters partagés, et de capter des structures connectées.

Cela permet à Mapper de connecter des clusters partiels pour reconstruire la forme globale (ex : boucle, branche, cluster étendu…)

<span style="color:orange; font-weight:bold">Clustering avec DBSCAN</span>

Pour chaque cube (sous-région), on applique DBSCAN :

- Un algorithme robuste aux formes irrégulières
- Pas besoin de spécifier le nombre de clusters à l’avance
- Utilise ici une distance de corrélation, adaptée à des séries temporelles (retours).

Paramètres importants :
- metric='correlation' : distance adaptée à des données de type rendement.
- eps=0.3 : seuil de proximité. Plus petit → plus de petits clusters.
- min_samples=3 : au moins 3 points pour former un cluster.

Sauvegarde visuelle du graph

In [ ]:
mapper.visualize(graph, path_html="portfolio.html")

Chaque nœud est un cluster local d’actions.

Si deux nœuds ont des actions en commun (à cause du recouvrement), on trace un lien (edge).

Résultat : un graphe de clusters connectés, qui représente la structure topologique des données.

In [16]:
nodes = graph['nodes']  # chaque noeud contient les index des stocks
clusters = [data_features.T.index[list(indices)] for indices in nodes.values()]


In [ ]:
cluster_target = 45

for i in range(len(clusters)):
    if i == cluster_target:
        sp_data[list(clusters[i])].cumsum().plot()
        


In [ ]:
cluster_target = 45

sp_data[list(clusters[cluster_target])].cumsum().plot(backend='plotly')

In [ ]:

sp_data[list(clusters[cluster_target])].sum(axis=1).cumsum().plot()

# Portolio  custom

In [21]:
df_infos = pd.read_csv("./sp500_gics_info_2010_2025.csv",index_col=0)

In [ ]:
ratio_arr = []
from collections import defaultdict

for i in range(len(clusters)):
    tmp = clusters[i]
    dic = defaultdict(int)
    for j in tmp:
        try:
            sector = df_infos.loc[j.split(" ")[0]][1]
        except:
            sector = "other"
        finally:
            dic[sector] += 1
    for j in dic:
        dic[j] /= len(tmp)
    ratio_arr.append((dic, i))

ratio_arr

In [42]:
def get_clusters_from_target(ratio_arr, target):

    def calculate_cluster_score(x):
        dic = x[0]
        ans = 0
        for i in target:
            ans += dic[i] * target[i]
        return ans

    ans = sorted(ratio_arr, key=calculate_cluster_score, reverse=1)
    return ans
target = {"Health Care": 1}

sah = get_clusters_from_target(ratio_arr, target)


In [ ]:
sah

In [ ]:
def print_raw_clustors_from_id_sorted(arr, index):

    sp_data[list(clusters[arr[index][1]])].cumsum().plot()


sash = print_raw_clustors_from_id_sorted(sah, 4)


In [ ]:
def print_agg_clustors_from_id_sorted(arr, index):

    sp_data[list(clusters[arr[index][1]])].sum(axis=1).cumsum().plot()


sash = print_agg_clustors_from_id_sorted(sah, 5)


In [88]:
sp_data[list(clusters[cluster_target])].cumsum().plot(backend='plotly').update_layout(width=1400).write_image(f"cluster_{cluster_target}.png")


In [ ]:
sp_data[list(clusters[cluster_target])].cumsum().plot(backend='plotly').update_layout(width=1400).write_html(f"cluster_{cluster_target}.html")


In [ ]:
sp_data[['AFL US Equity', 'AIG US Equity', 'AYI US Equity', 'ECL US Equity', 'HLT US Equity', 'HST US Equity', 'MNST US Equity', 'ODFL US Equity', 'PSA US Equity', 'R US Equity', 'WAT US Equity', 'YUM US Equity']].cumsum().plot(backend='plotly')

In [ ]:
def find_cluster_with_target(clusters)

# Resumé

L"zcrendements d’actions sont des points dans l’espace.
On les projette en 2D (UMAP). On pose un filet en carré sur cet espace (Cover).
Dans chaque carré, on trouve des amas denses (clusters) avec DBSCAN.
Puis on relie les amas qui se chevauchent, obtenant ainsi une carte de la topologie.

# Application

On essaiera de créer un portefeuille dont les poids refleteront le lien de chaque ticker avec les autres élements de cet univers.

Dans ce portfefeuille, on aura:
- Le capital est réparti équitablement entre les grands clusters (composantes connexes du graphe).
- Puis réparti entre les sous-clusters (nœuds) à l’intérieur de chaque grand cluster.
- Puis réparti entre les actions dans chaque sous-cluster.

C’est une stratégie de pondération hiérarchique.

On aura comme poids pour nos stocks

$$poids_{action}\ = \frac{1}{\text {nb clusters}} * \frac{1}{\text {nb sous-clusters dans cluster}} * \frac{1}{\text {nb actions dans sous clusters}}$$



## Exemple

In [ ]:
clustered_symbols = [

    [  # Grand cluster 1 (50%)
        ['AAPL', 'MSFT'],     # Sous-cluster 1 (25%)
        ['GOOG'],             # Sous-cluster 2 (25%)
    ],
    
    [  # Grand cluster 2 (50%)
        ['TSLA', 'NVDA'],     # Sous-cluster 3 (25%)
        ['AMZN', 'META'],     # Sous-cluster 4 (25%)
    ],

]

Weighting will be 

AAPL     0.125

MSFT     0.125

GOOG     0.25

TSLA     0.125

NVDA     0.125

AMZN     0.125

META     0.125

Cette fonction calcul le weighting recursivement

In [ ]:

def weight_distribution(clustered_symbols):
    weights = {}
    def assign_weights(nested_list, level=1):
        n = len(nested_list)
        w = 1 / n
        for item in nested_list:
            if isinstance(item, pd.Index):
                assign_weights(item, level + 1)
            else:
#                print(item)
                weights[item] = weights.get(item, 0) + w / (2 ** (level - 1))
    assign_weights(clustered_symbols)
    return pd.Series(weights) / sum(weights.values())


In [ ]:
weights = weight_distribution(clusters)
print(weights.sort_values(ascending=False).head(10))


In [ ]:
weights

In [ ]:
log_returns.columns

In [ ]:
for i in clusters:
    if len(i) == 17:
        print(type(i))
        print(isinstance(i, pd.Index))

In [ ]:
log_returns.T

In [ ]:
log_returns[weights.index] * weights

In [ ]:
tda_portfolio = log_returns[weights.index]
tda_portfolio = tda_portfolio * weights
tda_portfolio['sum_log_return'] = tda_portfolio.sum(axis=1)
tda_portfolio

In [ ]:
tda_portfolio['sum_log_return'].cumsum().plot()

In [ ]:
log_returns.sum(axis=1).cumsum().plot()

In [ ]:
def compute_sharp(strat):
    return strat.mean()/(strat.std() * np.sqrt(252) )
print(compute_sharp(log_returns.sum(axis=1)))
print(compute_sharp(tda_portfolio['sum_log_return']))
